<a href="https://colab.research.google.com/github/taichi0315/optimization-handson/blob/master/sample/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数理最適化ハンズオン


## 環境構築

In [2]:
!pip install pulp

     |████████████████████████████████| 40.6MB 114kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for amply: filename=amply-0.1.2-cp36-none-any.whl size=16572 sha256=64364a3d833efea42ebae48b2375188ce1d2957767adf1c94af4098a89717de2
  Stored in directory: /root/.cache/pip/wheels/84/18/f7/e5c3ed13ed5bb721763f77d4a924331d59ef115ce61c9d26eb
Successfully built amply


In [3]:
from pulp import *

## 集合
モデルを構成する要素・物体の集まり

- 職員
- シフト日付

In [4]:
# Set
Employee = { "emp1", "emp2", "emp3" }
Day       = { 1, 2, 3, 4, 5 }

In [5]:
print(Employee)

{'emp3', 'emp1', 'emp2'}


## 変数
最適化をして求めたい値。最適解が求まると変数が最適値として定まる。

- 保育士$e$が日付$d$に出勤するかどうか（0-1変数）
- 保育士$e$が超過ペナルティ日数を超過した日数



In [14]:
# Variable
x = {}
for emp in Employee:
    for day in Day:
        x[emp, day] = LpVariable(name=f"x_{emp}_{day}", cat="Binary") 

xp = {}
for emp in Employee:
    xp[emp] = LpVariable(name=f"xp_{emp}", lowBound=0, cat="Continuous")

In [15]:
print(xp)

{'emp3': xp_emp3, 'emp1': xp_emp1, 'emp2': xp_emp2}


## モデルの作成

In [ ]:
model = LpProblem(sense=LpMinimize)

## 制約
求解する上で必ず守る制約条件

- 職員の最低配置人数

In [ ]:
for day in Day:
    model.addConstraint(
        lpSum(x[emp, day] for emp in Employee)
        >=
        2
    )

for emp in Employee:
    model.addConstraint(
        xp[emp]
        ==
        lpSum(x[emp, day] for day in Day) - 3
    )

## 目的関数

何を最小化・最大化したいかを表す、最適化の目的となる関数

- 何日以上出勤したらペナルティ
- ペナルティの合計を最小化したい

In [ ]:
model += lpDot([xp[emp] for emp in Employee], [3] * len(Employee))

## 求解

In [ ]:
status = model.solve()
print(LpStatus[status])

Optimal


In [ ]:
for emp, day in x:
    print(emp, day, x[emp, day].value())

emp1 1 0.0
emp1 2 1.0
emp1 3 1.0
emp1 4 0.0
emp1 5 1.0
emp2 1 1.0
emp2 2 1.0
emp2 3 1.0
emp2 4 1.0
emp2 5 0.0
emp3 1 1.0
emp3 2 0.0
emp3 3 0.0
emp3 4 1.0
emp3 5 1.0


In [ ]:
for emp in Employee:
    print(xp[emp].value())

0.0
1.0
0.0
